# Data reduction workflow graph

We define a basic data reduction workflow using Sciline.
The individual steps are currently not functional (mostly empty dummy functions), but we can begin to define the workflow graph.

In [ ]:
from ess import polarization as pol
import scipp as sc
import sciline

## Opacity

There are two ways of computing the opacity, from cell parameters and from direct-beam measurements.

In [ ]:
workflow = sciline.Pipeline([pol.he3_opacity_from_cell_params])
workflow.visualize(pol.He3OpacityFunction[pol.Polarizer])

In [ ]:
workflow = sciline.Pipeline([pol.he3_opacity_from_beam_data])
workflow.visualize(pol.He3OpacityFunction[pol.Polarizer])

## Full pipeline

### On-the-fly reduction: opacity from cell parameters

#### With meta data from files

In [ ]:
from copy import copy

providers = copy(pol.providers)
providers.remove(pol.he3_opacity_from_beam_data)
providers.append(pol.he3_opacity_from_cell_params)
workflow = sciline.Pipeline(providers)
results = (pol.PolarizationCorrectedSampleData,)
graph = workflow.visualize(results, graph_attr={'rankdir': 'LR'})
graph.render('polarization-from-cell-params', format='png')
graph

#### Manual config

Notes:
- We do not have final file structure yet.
  Therefore we compute $I(Q)$ and direct beam manually and set it in the workflow.
- `he3_polarization` and `he3_transmission` are not implemented yet.

In [ ]:
workflow[pol.base.ReducedSampleDataBySpinChannel[pol.Down, pol.Down]] = None
workflow[pol.base.ReducedSampleDataBySpinChannel[pol.Up, pol.Down]] = None
workflow[pol.base.ReducedSampleDataBySpinChannel[pol.Down, pol.Up]] = None
workflow[pol.base.ReducedSampleDataBySpinChannel[pol.Up, pol.Up]] = None
workflow[pol.base.ReducedDirectBeamDataNoCell] = None
workflow[pol.base.ReducedDirectBeamData[pol.Analyzer,pol.Polarized]] = None
workflow[pol.base.ReducedDirectBeamData[pol.Polarizer,pol.Polarized]] = None
workflow.visualize(pol.He3Transmission[pol.Polarizer], graph_attr={'rankdir': 'LR'})

### Precise reduction: opacity from direct-beam measurements of unpolarized cells

In [ ]:
workflow = sciline.Pipeline(pol.providers)
results = (pol.PolarizationCorrectedSampleData,)
graph = workflow.visualize(results, graph_attr={'rankdir': 'LR'})
graph.render('polarization-from-neutron-beam', format='png')
graph